In [ ]:
import pandas as pd
import numpy as np
from cleaning_helpers import clean_column

In [ ]:
# Read Respnose Data
initial_resp_df = pd.read_csv('src/data/response_data/initial_responses.csv')
initial_resp_df.columns = ['resp_date', 
                           'landlord_name', 
                           'landlord_rating', 
                           'landlord_review', 
                           'resp_address', 
                           'total_rent', 
                           'utilities', 'parking', 
                           'property_rating', 
                           'ind_rent', 
                           'property_review']

In [ ]:
# Rename Utility Responses
initial_resp_df.loc[initial_resp_df['utilities'].str.contains('Some'), 'utilities'] = 'Some'
initial_resp_df.loc[initial_resp_df['utilities'].str.contains('No'), 'utilities'] = 'None'
initial_resp_df.loc[initial_resp_df['utilities'].str.contains('All'), 'utilities'] = 'All'

In [21]:
# Clean address column
initial_resp_df['resp_address'] = clean_column(initial_resp_df, 'resp_address')

In [ ]:
# Clean addresses
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('street','st')
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('court', 'ct')
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('avenue', 'ave')
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('circle', 'cir')
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('road', 'rd')
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('place', 'pl')
initial_resp_df['resp_address']= initial_resp_df['resp_address'].str.replace('lane', 'ln')
initial_resp_df['resp_address'] = initial_resp_df['resp_address'].str.split('williamsburg').str[0]

# Fix incorrect addresses
initial_resp_df['resp_address'] = initial_resp_df['resp_address'].str.replace('221monticelloave', '215monticelloave')
initial_resp_df['resp_address'] = initial_resp_df['resp_address'].str.replace('201monticelloave', '215monticelloave')
initial_resp_df['resp_address'] = initial_resp_df['resp_address'].str.replace('220northboundary', '501princegeorgest')
initial_resp_df['resp_address'] = initial_resp_df['resp_address'].str.replace('1210richmondrdinto8', '1210richmondrdunit8')
initial_resp_df['resp_address'] = initial_resp_df['resp_address'].str.replace('idontcurrentlylivein711richmondrdbutididayearago', '711richmondrd')


In [ ]:
# initial_resp_df[['PID', 'address']]
property_df = pd.read_csv('src/data/tProperty.csv')

In [24]:
merged = property_df.merge(
    initial_resp_df,
    left_on='address',
    right_on='resp_address',
    how='left'
)
cols = list(initial_resp_df.columns)
cols.append('pid')
resp_valid_address = merged[cols].dropna(how='all',subset=initial_resp_df.columns).reset_index(drop=True)

In [25]:
resp_valid_address = resp_valid_address.drop(columns='resp_address')

In [26]:
resp_valid_address.loc[resp_valid_address['landlord_review'].isna(), 'landlord_review'] = ''
resp_valid_address.loc[resp_valid_address['property_review'].isna(), 'property_review'] = ''

# For purposes of sample data set property_rating to landlord rating where property_rating is nan (not asked on original survey)
resp_valid_address.loc[resp_valid_address['property_rating'].isna(), 'property_rating'] = resp_valid_address.loc[resp_valid_address['property_rating'].isna(), 'landlord_rating']
resp_valid_address.loc[resp_valid_address['ind_rent'].isna() & (resp_valid_address['landlord_name']=='Current Midtown'), 'ind_rent'] = '1000 - 1,500'

In [34]:
resp_valid_address['resp_date'] = resp_valid_address['resp_date'].str.split(' ').str[0]

In [ ]:
resp_valid_address.to_csv('src/data/tResponse.csv')

In [36]:
resp_valid_address

,resp_date,landlord_name,landlord_rating,landlord_review,total_rent,utilities,parking,property_rating,ind_rent,property_review,pid
0,5/9/2025,Gary shelly,1.0,Gary acts like he is a nice caring man but in ...,"4,500-5,500",None,Yes,2.0,"500 - 1,000",The house was big and had the potential to be ...,2146
1,5/10/2025,Brooks Real Estate,4.0,Very responsive when we submit service request...,"2,500 - 3,500",None,No,2.0,"1000 - 1,500",Our house is rat infested due to years of negl...,2149
2,4/30/2025,Greg Granger,3.0,"Rude and raises rent, but chill with some things","4,500-5,500",None,Yes,5.0,"1000 - 1,500",Super spacious and great location,2569
3,5/8/2025,Williamsburg Rentals - Ashley Ribock,3.0,,"2,500 - 3,500",None,Yes,3.0,"500 - 1,000",,1081
4,5/8/2025,Greg Granger,5.0,"He’s very receptive, every problem we have is ...","2,500 - 3,500",None,No,2.0,"500 - 1,000",,1080
5,5/9/2025,Keith,3.0,"He doesn’t bother us, however, he also never f...","2,500 - 3,500",Some,Yes,3.0,< 500,It’s a frat house for sure.,2203
6,5/8/2025,Agathos Inc. (Becky Hilstrom),2.0,"Takes a very hands off approach, doesn’t commu...","3,500 - 4,500",None,Yes,3.0,"500 - 1,000",,4204
7,4/29/2025,Tom and Jan Barry,1.0,Despite a written rule in our housing contract...,"3,500 - 4,500",Some,Yes,4.0,"500 - 1,000",,1119
8,5/8/2025,Colby,1.0,Horrible slum lord and overall bad peraon,"2,500 - 3,500",None,No,1.0,"500 - 1,000",,4294
9,4/16/2025,Current Midtown,3.0,Overpriced and exploitative but otherwise fine,"500 - 1,500",All,No,3.0,"1000 - 1,500",,82058
